compare calibration result and simulation result in the instant loop detector

In [225]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'w2e_in'
number = 19
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [226]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [227]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,w2e_in,1.577837e+09,enter,0_w2e_in,9.05,5.00,DEFAULT_VEHTYPE,None
2,w2e_in,1.577837e+09,enter,1_w2e_in,9.05,5.00,DEFAULT_VEHTYPE,2.39
4,w2e_in,1.577837e+09,enter,2_w2e_in,8.44,5.00,DEFAULT_VEHTYPE,1.58
6,w2e_in,1.577837e+09,enter,3_w2e_in,10.45,5.00,DEFAULT_VEHTYPE,9.25
8,w2e_in,1.577837e+09,enter,4_w2e_in,11.20,5.00,DEFAULT_VEHTYPE,20.52
10,w2e_in,1.577837e+09,enter,5_w2e_in,10.46,5.00,DEFAULT_VEHTYPE,2.89
12,w2e_in,1.577837e+09,enter,6_w2e_in,10.00,5.00,DEFAULT_VEHTYPE,29.08
15,w2e_in,1.577837e+09,enter,7_w2e_in,11.47,5.00,DEFAULT_VEHTYPE,3.65
17,w2e_in,1.577837e+09,enter,8_w2e_in,9.58,5.00,DEFAULT_VEHTYPE,21.59
19,w2e_in,1.577837e+09,enter,9_w2e_in,9.04,5.00,DEFAULT_VEHTYPE,25.50


In [228]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_in,1.577837e+09,8.950380,1.196952,1577836811,1577836766,max,9.166667,0.13,-0.216286
1,1_w2e_in,1.577837e+09,8.504859,1.174561,1577836814,1577836768,max,7.500000,0.14,1.004859
2,2_w2e_in,1.577837e+09,9.195337,1.209466,1577836815,1577836769,max,8.888889,-0.43,0.306448
3,3_w2e_in,1.577837e+09,11.019084,1.410434,1577836826,1577836788,max,11.111111,0.10,-0.092027
4,4_w2e_in,1.577837e+09,10.250646,1.382051,1577836847,1577836808,max,7.777778,0.11,2.472868


In [229]:
print(len(data), len(df_instant_out))

19 19


compare time error and speed error in loop detector between simulation result and calibration result

In [230]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_w2e_in,1.577837e+09,9.05,1.577837e+09,8.950380,1577836811,9.166667
1,1_w2e_in,1.577837e+09,9.05,1.577837e+09,8.504859,1577836814,7.500000
2,2_w2e_in,1.577837e+09,8.44,1.577837e+09,9.195337,1577836815,8.888889
3,3_w2e_in,1.577837e+09,10.45,1.577837e+09,11.019084,1577836826,11.111111
4,4_w2e_in,1.577837e+09,11.20,1.577837e+09,10.250646,1577836847,7.777778


In [231]:
#compare_df = compare_df.head(19)

In [232]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.38105266972592)

In [233]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.4615789463645534)

In [234]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(1.0442851481864543)

In [235]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(1.3525146198830411)

In [236]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}_new.html")

In [237]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-0.3399999141693115 2.180000066757202
-0.5199999809265137 2.130000114440918


In [238]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-0.6611111111111114 3.9700000000000006
-0.2162862497209872 3.044843208533038


In [239]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}_new.html")

In [240]:
#import os
#os.environ["HORNSGATAN_HOME"]
